In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 6

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(4):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
33206,"[2054.0625, 729.9002685546875]","[2053.677734375, 729.1788330078125]","[2052.819091796875, 728.55419921875]","[2052.203125, 727.9384155273438]","[2051.5908203125, 727.2872314453125]","[2050.9365234375, 726.6475830078125]","[2050.272705078125, 726.00048828125]","[2049.589599609375, 725.3515014648438]","[2048.892333984375, 724.70068359375]","[2048.18310546875, 724.0488891601562]",...,"[2040.117919921875, 716.8460083007812]","[2039.38134765625, 716.1786499023438]","[2038.6435546875, 715.50634765625]","[2037.9036865234375, 714.82861328125]","[2037.1611328125, 714.1451416015625]","[2036.415283203125, 713.4559936523438]","[2035.665771484375, 712.7616577148438]","[2034.91259765625, 712.0634765625]","[2034.15625, 711.3634643554688]","[2033.3978271484375, 710.664794921875]"
25274,"[238.42422485351562, 1390.59326171875]","[238.63619995117188, 1389.65771484375]","[238.41656494140625, 1388.7349853515625]","[238.53610229492188, 1387.967041015625]","[238.5830535888672, 1387.1915283203125]","[238.6302490234375, 1386.3902587890625]","[238.66798400878906, 1385.5802001953125]","[238.69729614257812, 1384.7601318359375]","[238.71986389160156, 1383.9334716796875]","[238.73672485351562, 1383.101806640625]",...,"[238.8643798828125, 1373.884521484375]","[238.89263916015625, 1373.042236328125]","[238.9263153076172, 1372.197021484375]","[238.9658203125, 1371.3477783203125]","[239.01156616210938, 1370.493408203125]","[239.06393432617188, 1369.6329345703125]","[239.12326049804688, 1368.7652587890625]","[239.1898956298828, 1367.8892822265625]","[239.26412963867188, 1367.0037841796875]","[239.34625244140625, 1366.1080322265625]"
8026,"[727.1561279296875, 1538.4156494140625]","[727.3751220703125, 1537.99658203125]","[727.11572265625, 1537.2904052734375]","[727.1748046875, 1536.7843017578125]","[727.1715087890625, 1536.2403564453125]","[727.1668701171875, 1535.6966552734375]","[727.1544799804688, 1535.1519775390625]","[727.1343994140625, 1534.6068115234375]","[727.107666015625, 1534.0625]","[727.0748291015625, 1533.51953125]",...,"[726.58544921875, 1527.6156005859375]","[726.552978515625, 1527.0740966796875]","[726.52587890625, 1526.529052734375]","[726.504638671875, 1525.97998046875]","[726.4899291992188, 1525.42626953125]","[726.4822387695312, 1524.8671875]","[726.4821166992188, 1524.30224609375]","[726.4901123046875, 1523.730712890625]","[726.5067138671875, 1523.1519775390625]","[726.532470703125, 1522.5653076171875]"
7049,"[1826.0928955078125, 433.3138732910156]","[1826.962890625, 434.0237731933594]","[1827.739013671875, 434.5923156738281]","[1828.511962890625, 435.22161865234375]","[1829.2757568359375, 435.8701171875]","[1830.0340576171875, 436.53143310546875]","[1830.7896728515625, 437.2019958496094]","[1831.5450439453125, 437.8787841796875]","[1832.30224609375, 438.5596008300781]","[1833.0625, 439.24298095703125]",...,"[1841.71728515625, 446.79803466796875]","[1842.52490234375, 447.47674560546875]","[1843.3348388671875, 448.1483154296875]","[1844.1473388671875, 448.809814453125]","[1844.9625244140625, 449.4579772949219]","[1845.7806396484375, 450.08917236328125]","[1846.60205078125, 450.699462890625]","[1847.42724609375, 451.2848205566406]","[1848.2564697265625, 451.8411865234375]","[1849.0902099609375, 452.36456298828125]"
19801,"[587.6060180664062, 1113.4151611328125]","[587.7677001953125, 1113.2283935546875]","[587.7426147460938, 1112.9239501953125]","[587.8157348632812, 1112.5926513671875]","[587.8928833007812, 1112.266357421875]","[587.9656372070312, 1111.940673828125]","[588.0375366210938, 1111.621337890625]","[588.1044311523438, 1111.3074951171875]","[588.16650390625, 1111.0001220703125]","[588.2234497070312, 1110.69921875]",...,"[588.67578125, 1107.75634765625]","[588.72021484375, 1107.5113525390625]","[588.7685546875, 1107.2679443359375]","[588.8214111328125, 1107.025634765625]","[588.8796997070312, 1106.783935546875]","[588.944091796875, 1106.5426025390625]","[589.015380859375, 11

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
33206,2054.062500,729.900269,2053.677734,729.178833,2052.819092,728.554199,2052.203125,727.938416,2051.590820,727.287231,...,2036.415283,713.455994,2035.665771,712.761658,2034.912598,712.063477,2034.156250,711.363464,2033.397827,710.664795
25274,238.424225,1390.593262,238.636200,1389.657715,238.416565,1388.734985,238.536102,1387.967041,238.583054,1387.191528,...,239.063934,1369.632935,239.123260,1368.765259,239.189896,1367.889282,239.264130,1367.003784,239.346252,1366.108032
8026,727.156128,1538.415649,727.375122,1537.996582,727.115723,1537.290405,727.174805,1536.784302,727.171509,1536.240356,...,726.482239,1524.867188,726.482117,1524.302246,726.490112,1523.730713,726.506714,1523.151978,726.532471,1522.565308
7049,1826.092896,433.313873,1826.962891,434.023773,1827.739014,434.592316,1828.511963,435.221619,1829.275757,435.870117,...,1845.780640,450.089172,1846.602051,450.699463,1847.427246,451.284821,1848.256470,451.841187,1849.090210,452.364563
19801,587.606018,1113.415161,587.767700,1113.228394,587.742615,1112.923950,587.815735,1112.592651,587.892883,1112.266357,...,588.944092,1106.542603,589.015381,1106.301270,589.094238,1106.059692,589.181519,1105.817627,589.277954,1105.575195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12998,2186.586182,731.518066,2186.368896,730.924133,2185.900635,730.666931,2185.518799,730.173340,2185.100342,729.667542,...,2174.623535,718.258484,2174.090088,717.706909,2173.557861,717.152832,2173.026855,716.596191,2172.496826,716.037292
32750,2075.543213,633.811035,2076.241943,635.185791,2078.076172,636.469055,2079.831055,637.765930,2081.637939,639.025513,...,2117.099365,666.914978,2118.246826,668.219116,2118.793457,668.932556,2118.794189,669.105225,2118.474365,668.980652
9101,1867.209473,464.413391,1868.436035,465.583862,1869.643799,466.540558,1870.873657,467.559021,1872.112427,468.585480,...,1897.468262,489.122192,1898.553711,489.918152,1899.622559,490.678802,1900.676880,491.408203,1901.721191,492.114960


In [12]:
df2.to_csv("outputs6ep.csv", index=True)